### Récupération des données

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import time

df = pd.read_csv('/content/drive/MyDrive/horsesData.csv')

data = df[['raceId', 'horse_genyId','horse_name','results_position']]
data

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (31,32,47,48,57,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,raceId,horse_genyId,horse_name,results_position
0,pmu_race:01012017/R1/C10,2421081,CALIE DU TROT,NaN
1,pmu_race:01012017/R1/C10,2449264,COLANIE,NaN
2,pmu_race:01012017/R1/C10,2433769,CHARMANTE LULU,NaN
3,pmu_race:01012017/R1/C10,2480332,CAQUISE DE NGANDA,1.0
4,pmu_race:01012017/R1/C10,2410966,CATOWICE DE CLECY,4.0
...,...,...,...,...
16579,pmu_race:31012017/R4/C8,2376906,LADY FORD,9.0
16580,pmu_race:31012017/R4/C8,2290730,SUPER TOFT,7.0
16581,pmu_race:31012017/R4/C8,2361646,MEILO MO,2.0
16582,pmu_race:31012017/R4/C8,2144544,SEDUS,8.0


### Création tableau des classements elo

In [ ]:
classement = pd.DataFrame(columns = ['id', 'nom', 'elo']).set_index('id', drop=False)
classement

,id,nom,elo
id,,,


### Code classement elo

In [ ]:
########## FONCTION ########## Code modification elo #########

def new_elo(anciens_elo, resultats):
    """
    Calcule la liste des nouveaux elos
    Args :
      anciens_elo -- liste de float
      resultats -- liste de float, contenant des nan (et rien d'autre)
    """
    K = 24   #Paramètre pour le calcul
    nb_partants = len(anciens_elo)  
    nb_arrivees = np.count_nonzero(~np.isnan(resultats))
    
    #Un pti mot pour dire ce qu'est R ?
    R = 10**(anciens_elo/400)
    Rtot = np.sum(R)

    E = R/(R+(1/(nb_partants-1))*(Rtot-R))

    S = np.array([(nb_arrivees-position+1)/nb_arrivees if not np.isnan(position) else 0 for position in resultats])
    
    new_elo_tot = K*np.sum(S-E)
    nvx_elo = anciens_elo + K*(S-E) - new_elo_tot/nb_partants
    
    return nvx_elo

In [ ]:
def maj(classement, ids, elos):
    for indice, id in enumerate(ids):
      classement.at[id, 'elo'] = elos[indice]
pd.set_option('display.max_columns', None)

Dans un premier temps, nous initialisons l'elo de tous les chevaux qui ne sont pas dans la database classement.

In [ ]:
tmps1=time.clock()
ELO_INITIAL = 1500

#On travaille d'abord sur des numpy arrays (plus rapide et plus léger)
index_absents = ~data['horse_genyId'].isin(classement['id']).values  
absents = data[['horse_genyId','horse_name']][index_absents].values
liste_elo = np.broadcast_to(ELO_INITIAL, (len(absents), 1))  #on crée une liste de la bonne taille avec les élos initiaux
nvx_chevaux_data = np.concatenate((absents, liste_elo), axis=1)  #on crée la "base" du dataframe : colonnes = id, nom, elo


#Actualisation du dataframe de référence
df2 = pd.DataFrame(nvx_chevaux_data, columns = ['id', 'nom', 'elo'])
classement = classement.append(df2, ignore_index=True).set_index('id', drop=False)

classement.drop_duplicates(inplace=True, subset=['id'])  #On a ajouté les chevaux à chaque fois qu'ils apparaissaient dans le fichier course 
                                                          #et qu'ils n'étaient pas dans le classement. Mais ils peuvent courir plusieurs fois

#Parcours successif des courses pour calculer les nouveaux elos 
g = data.groupby('raceId')

for _, race_data in g:
    #On récupère dans un df toutes les données dont on a besoin avec une jointure à gauche (tous les chevaux de la course observée)
    infos_course = pd.merge(race_data, classement, how='inner', left_on='horse_genyId', right_index=True)

    nouveaux_elos = new_elo(infos_course['elo'].values, infos_course['results_position'].values)
    maj(classement, infos_course['horse_genyId'].values, nouveaux_elos)

print(classement)
tmps2=time.clock()
print(tmps2-tmps1)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


             id     nom      elo
count     11574   11574  11574.0
unique    11574   11551     15.0
top     2457599  DANCER   1500.0
freq          1       2  11559.0
              id                nom      elo
id                                          
2421081  2421081      CALIE DU TROT  1494.46
2449264  2449264            COLANIE  1490.24
2433769  2433769     CHARMANTE LULU  1487.32
2480332  2480332  CAQUISE DE NGANDA  1535.47
2410966  2410966  CATOWICE DE CLECY  1486.95
...          ...                ...      ...
2281507  2281507        TUTANCHAMON  1505.39
2250735  2250735       VIEUX MANOIR  1492.99
2444735  2444735        DUKE ELAINE  1496.41
2500277  2500277              MILES  1498.81
2481846  2481846          KIMBOY MO  1501.21

[11574 rows x 3 columns]
5.188583999999992


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
